In [2]:
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd
from plotly import express as px
from plotly import graph_objects as go
from scipy.stats import norm, chi2, shapiro, kstest, ks_2samp
import math
iris = pd.DataFrame(data= np.c_[load_iris()['data'], load_iris()['target']],
                     columns= load_iris()['feature_names'] + ['target'])

iris50 = iris.iloc[0:50,0:4]
iris50.head(10)
iris50_array = np.matrix(iris50)                    
  

In [5]:
def QQ_Plot(data, what_are_you_looking_at):
    sorted_data = np.sort(np.array(data))
    j_terms = np.array(range(1,len(sorted_data)+1))
    converted_values = [norm.ppf((j - 0.5)/(len(sorted_data))) for j in j_terms]
    
    correlation_coef = (np.corrcoef(sorted_data, converted_values))[0,1]

    slope_of_line = ( converted_values[math.ceil(len(sorted_data)*.75)] - converted_values[math.ceil(len(sorted_data)*.25)] ) / ( sorted_data[math.ceil(len(sorted_data)*.75)] - sorted_data[math.ceil(len(sorted_data)*.25)] )
    QQline = lambda x : slope_of_line * (x - sorted_data[math.ceil(len(sorted_data)*.5)]) + converted_values[math.ceil(len(sorted_data)*.5)]

    fig = go.Figure(
        data = go.Scatter(
            x = sorted_data,
            y = converted_values,
            mode = 'markers'
        )
    )
    fig.add_shape(type='line',
                x0=(sorted_data[0] - .2),
                y0=QQline((sorted_data[0] - .2)),
                x1=(sorted_data[-1] + .2),
                y1=QQline((sorted_data[-1] + .2)),
                line=dict(color='Red',),
                xref='x',
                yref='y'
                )
    fig.update_layout(
        title = go.layout.Title(
            text = f"""Quantile-Quantile Plot of {what_are_you_looking_at}<br><sup>Slope of line is calculated from 75 and 25 percentitle values. Line runs through median of sorted data.""",
            xref = "paper",
            x = 0
        ),
        xaxis_title = "Theoretical Quantiles",
        yaxis_title = "Normal Distribution Quantiles"
    )
    fig.add_annotation(
        x=sorted_data[math.floor(len(sorted_data)*.5)],
        y=QQline((sorted_data[-1] + .2)),
        text = f"""Correlation Coefficient between Theoretical Quantiles and Normal Distribution Quantiles: {round(correlation_coef,5)}""",
        showarrow= False,
        xshift=10

    )
    fig.show()


def Chi_Squared_Test(data,what_are_you_looking_at):
    n = data.shape[0]
    p = data.shape[1]
    mu = np.matrix([round(np.mean(data[:,x]),5) for x in range(p)])
    variance_cov = data.transpose()*(np.identity(n) - (1/n)*np.ones(n))*data    
    mahalanobis =  np.array([float(np.matmul(np.matmul((data[x,:] - mu),np.linalg.inv(variance_cov)),(data[x,:] - mu).transpose())) for x in range(n)])

    sorted_data = np.sort(np.array(mahalanobis))
    j_terms = np.array(range(1,len(sorted_data)+1))
    converted_values = [chi2.ppf((j - 0.5)/(len(sorted_data)),df = p) for j in j_terms]

    correlation_coef = (np.corrcoef(sorted_data, converted_values))[0,1]    

    slope_of_line = ( converted_values[math.ceil(len(sorted_data)*.75)] - converted_values[math.ceil(len(sorted_data)*.25)] ) / ( sorted_data[math.ceil(len(sorted_data)*.75)] - sorted_data[math.ceil(len(sorted_data)*.25)] )
    QQline = lambda x : slope_of_line * (x - sorted_data[math.ceil(len(sorted_data)*.5)]) + converted_values[math.ceil(len(sorted_data)*.5)]


    fig = go.Figure(
        data = go.Scatter(
            x = sorted_data,
            y = converted_values,
            mode = 'markers'
        )
    )

    fig.add_shape(type='line',
                x0=(sorted_data[0] - .1),
                y0=QQline((sorted_data[0] - .1)),
                x1=(sorted_data[-1] + .1),
                y1=QQline((sorted_data[-1] + .1)),
                line=dict(color='Red',),
                xref='x',
                yref='y'
                )
    fig.update_layout(
        title = go.layout.Title(
            text = f"""Chi-Squared Quantile-Quantile Plot of {what_are_you_looking_at}<br><sup>Slope of line is calculated from 75 and 25 percentitle values. Line runs through median of sorted squared mahalanobis distance data.""",
            xref = "paper",
            x = 0
        ),
        xaxis_title = "Squared Mahalanobis Distance",
        yaxis_title = "Chi-Squared Quantile"
    )
    fig.add_annotation(
    x=sorted_data[math.floor(len(sorted_data)*.5)],
    y=QQline((sorted_data[-1] + .2)),
    text = f"""Correlation Coefficient between Theoretical Quantiles and Normal Distribution Quantiles: {round(correlation_coef,5)}""",
    showarrow= False,
    xshift=10)

    
    return fig, converted_values


In [6]:
param_titles = {
    "Sepal Length": "sepal length (cm)",
    "Sepal Width": "sepal width (cm)",
    "Petal Length": "petal length (cm)",
    "Petal Width": "petal width (cm)"
}

for why_did_i_master_in_math in param_titles.keys():
    QQ_Plot(iris50[param_titles[why_did_i_master_in_math]], why_did_i_master_in_math)
fig, cv = Chi_Squared_Test(iris50_array, "first 50 iris data entries")
fig


In [317]:
param_titles = [
"sepal length (cm)",
"sepal width (cm)",
"petal length (cm)",
"petal width (cm)"
]
for x in param_titles:
    print(f"""Shapiro-Wilk test for {x}:\nTest statistic: {(shapiro(iris50[x]))[0]}\nP-Value: {(shapiro(iris50[x]))[1]}\n""")

Shapiro-Wilk test for sepal length (cm):
Test statistic: 0.9776989221572876
P-Value: 0.4595281183719635

Shapiro-Wilk test for sepal width (cm):
Test statistic: 0.97171950340271
P-Value: 0.2715264856815338

Shapiro-Wilk test for petal length (cm):
Test statistic: 0.9549766182899475
P-Value: 0.05481043830513954

Shapiro-Wilk test for petal width (cm):
Test statistic: 0.7997642159461975
P-Value: 8.65842082475865e-07



Sepal Length and Sepal Width both have relatively high p-values. Due to this, we have little to no evidence to reject the null hypothesis that the data for sepal length and width come from a normal distribution. The p-value for the data of petal width is right around .05 (usually 95% is a standard confidence level). With this, we have some reason to reject the null hypothesis that the data for petal length comes from a normal distribution, but since we are still within our confidence level, it isn't fully clear to reject the null hypothesis. Further testing is needed. The p-value for the SW test of petal width is indicating that we should reject the null hypothesis of petal width comes from a normal distribution.

In [379]:
comparison_data = norm.rvs(loc = 5.006, scale = np.sqrt(0.121764), size = 100000, random_state = 42)
print(f"""
Kolmogorov-Smirnov test comparing the distributions of the Sepal length data and 
random variables from a N(5.006, 0.121764)

P-Value: {ks_2samp(iris50['sepal length (cm)'], comparison_data)[1]}""")


Kolmogorov-Smirnov test comparing the distributions of the Sepal length data and 
random variables from a N(5.006, 0.121764)

P-Value: 0.5039457679148336


In [385]:
comparison_data = norm.rvs(loc = 5.006, scale = np.sqrt(0.121764), size = 100000, random_state = 42)
print(f"""
Kolmogorov-Smirnov test comparing the distributions of the Sepal length data and 
random variables from a N(5.006, 0.121764)

P-Value: {ks_2samp(iris50['sepal length (cm)'], comparison_data)[1]}\n""")

comparison_data = norm.rvs(loc = 3.428, scale = np.sqrt(0.140816), size = 100000, random_state = 42)
print(f"""
Kolmogorov-Smirnov test comparing the distributions of the Sepal width data and 
random variables from a N(3.428, 0.140816)

P-Value: {ks_2samp(iris50['sepal width (cm)'], comparison_data)[1]}\n""")


comparison_data = norm.rvs(loc = 1.462, scale = np.sqrt(0.029556), size = 100000, random_state = 42)
print(f"""
Kolmogorov-Smirnov test comparing the distributions of the petal length data and 
random variables from a N(1.462, 0.029556)

P-Value: {ks_2samp(iris50['petal length (cm)'], comparison_data)[1]}\n""")

comparison_data = norm.rvs(loc = 0.246, scale = np.sqrt(0.010884), size = 100000, random_state = 42)
print(f"""
Kolmogorov-Smirnov test comparing the distributions of the petal width data and 
random variables from a N(0.246, 0.010884)

P-Value: {ks_2samp(iris50['petal width (cm)'], comparison_data)[1]}\n""")



Kolmogorov-Smirnov test comparing the distributions of the Sepal length data and 
random variables from a N(5.006, 0.121764)

P-Value: 0.5039457679148336


Kolmogorov-Smirnov test comparing the distributions of the Sepal width data and 
random variables from a N(3.428, 0.140816)

P-Value: 0.6186422417060236


Kolmogorov-Smirnov test comparing the distributions of the petal length data and 
random variables from a N(1.462, 0.029556)

P-Value: 0.17710719354341542


Kolmogorov-Smirnov test comparing the distributions of the petal width data and 
random variables from a N(0.246, 0.010884)

P-Value: 5.503252159087047e-06



My last test looking at the normality of the iris data comes in the way of looking at p-values of two sample Kolmogorov-Smirnov tests. With this, I compared the distributions of  each parameter (sepal length, sepal width, etc) to random variables from distributions with means and variances composed their MLE estimations. When I ran the tests, I got the p-values above. 

The two-sample Kolmogorov-Smirnov test's null hypothesis is that the two distributions are identical, or F(x) = G(x) for all x. With this in mind, we have little to no evidence to refute the null hypothesis for sepal length and width. Again, the p-value associated with petal length is lower compared to it's sepal length and width counterparts, but its well within a 95% confidence level. Lastly, we can see that p-value associated with petal width is very low, showing strong evidence to reject the null hypothesis. 

The two-sample KS test may not be a bullet proof test in terms of general univariate normality. My reasoning for this is just because petal width's distribution isn't identical to N(0.246, 0.010884), doesn't mean that we couldn't find different $\mu$ and $\sigma^2$ such that we fail to reject the null hypothesis of a two-sample KS test. Though, with the previous evidence from the Shapiro-Wilk test as well as the Normal Quantile-Quantile plots, I am confident in saying that the petal-width values of the first 50 values of the iris data is not from Normal distribution. 

Moving forward, I would look to do some sort of transformation with the petal width and maybe even petal length for testing purposes.

In [393]:
x1 = np.array(iris50_array[:,0])
x2 = iris50_array[:,1]
x3 = iris50_array[:,2]
x4 = iris50_array[:,3]
fig = go.Figure()
fig.add_trace(go.Box(
    y = iris50['sepal length (cm)'],
    name = 'Sepal Length'
))
fig.add_trace(go.Box(
    y = iris50['sepal width (cm)'],
    name = 'Sepal Width'
))
fig.add_trace(go.Box(
    y = iris50['petal length (cm)'],
    name = 'Petal Length'
))
fig.add_trace(go.Box(
    y = iris50['petal width (cm)'],
    name = 'Petal Width'
))
fig.update_layout(
    yaxis_title='Length in centimeters')



Just by creating box plots for each parameter, we could see that we might have two upper outliers for petal width, and one lower outlier for Sepal Width and Petal length respectively. More testing needs to be done to confirm. You can zoom in the widget.

In [447]:
df = pd.DataFrame()
param_list = ['sepal length (cm)', "sepal width (cm)",  "petal length (cm)" , "petal width (cm)"]
MLEMU = [5.006,3.428,1.462,0.246]
MLE_Sigma = [np.sqrt(0.121764),np.sqrt(0.140816),np.sqrt(0.029556),np.sqrt(0.010884)]


for y in range(4):
    absolute_z_scores = []
    absolute_z_scores = [np.abs(x - MLEMU[y])/ MLE_Sigma[y] for x in iris50[param_list[y]]]
    print(f"""Possible outliers in {param_list[y]}: {len([i for i in absolute_z_scores if i > 3])}""")

    

Possible outliers in sepal length (cm): 0
Possible outliers in sepal width (cm): 1
Possible outliers in petal length (cm): 0
Possible outliers in petal width (cm): 1
